In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from resnet_rn06_eembc import * 
import qkeras
!pwd
model = qkeras.utils.load_qmodel('resnet_v1_eembc_RN06_bilinear/small_model_best.h5')
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score, accuracy_score, top_k_accuracy_score
batch_size=32

2022-09-29 17:03:58.379287: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-29 17:03:58.681471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-29 17:03:58.681513: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/bhawks/RN07-Pokemon


2022-09-29 17:04:26.923268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-29 17:04:26.937469: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-29 17:04:26.937591: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-bhawks-poke): /proc/driver/nvidia/version does not exist
2022-09-29 17:04:26.950383: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        validation_split=0.25,
        rescale=1./255 # normalize values to between 0-1
        # preprocessing_function=random_crop,
        #brightness_range=(0.9, 1.2),
        #contrast_range=(0.9, 1.2)
    )
    tdatagen = ImageDataGenerator(rescale=1./255)
    
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        validation_split=0.25,
        rescale=1./255 # normalize values to between 0-1
        # preprocessing_function=random_crop,
        #brightness_range=(0.9, 1.2),
        #contrast_range=(0.9, 1.2)
    )
    tdatagen = ImageDataGenerator(rescale=1./255)
    
    
    train_generator = datagen.flow_from_directory('./data/SmallPokemonData/',
                                target_size=(32,32),
                                batch_size=batch_size,
                                color_mode='rgb',
                                class_mode='categorical',
                                interpolation='bilinear',
                                subset='training',
                                keep_aspect_ratio=True,                                             
                                )
    
    validation_generator = datagen.flow_from_directory('./data/SmallPokemonData/',
                                target_size=(32,32),
                                batch_size=batch_size,
                                color_mode='rgb',
                                class_mode='categorical',
                                interpolation='bilinear',
                                subset='validation',
                                keep_aspect_ratio=True
                                )    
    test_generator = datagen.flow_from_directory('./data/PokeCard/',
                            target_size=(32,32),
                            batch_size=1,
                            color_mode='rgb',
                            class_mode='categorical',
                            interpolation='bilinear',
                            shuffle=False)#,
                            #keep_aspect_ratio=True)
    
    test2_generator = tdatagen.flow_from_directory('./data/PokeCard/',
                            target_size=(32,32),
                            batch_size=batch_size,
                            color_mode='rgb',
                            class_mode='categorical',
                            interpolation='bilinear',
                            shuffle=False)#,
                            #keep_aspect_ratio=True)

Found 2002 images belonging to 10 classes.
Found 665 images belonging to 10 classes.
Found 532 images belonging to 10 classes.
Found 532 images belonging to 10 classes.


In [3]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 q_conv2d_batchnorm (QConv2D  (None, 8, 8, 32)         1025      
 Batchnorm)                                                      
                                                                 
 q_activation (QActivation)  (None, 8, 8, 32)          0         
                                                                 
 q_conv2d_batchnorm_1 (QConv  (None, 8, 8, 32)         9377      
 2DBatchnorm)                                                    
                                                                 
 q_activation_1 (QActivation  (None, 8, 8, 32)         0         
 )                                                               
                                                             

In [31]:
score = model.evaluate(test_generator)
print(score)

17/17 [==============================] - 1s 78ms/step - loss: 2.1585 - accuracy: 0.6165
[2.158482074737549, 0.61654132604599]


In [67]:
data_list = []
batch_index = 0
while batch_index <= test_generator.batch_index:
    data = test_generator.next()
    data_list.append(data[0])
    batch_index = batch_index + 1
print(len(data_list))

532


In [32]:
score2 = model.evaluate(test2_generator)
print(score2)

17/17 [==============================] - 1s 54ms/step - loss: 1.9077 - accuracy: 0.6560
[1.9077112674713135, 0.6560150384902954]


In [61]:
    y_pred = model.predict(test_generator)
    # evaluate with test dataset and share same prediction results
    evaluation = model.evaluate(test_generator)
    y_test = test_generator.classes
    print(y_pred[0])
    print(y_test)
    score = top_k_accuracy_score(y_test, y_pred, k=2)
    print(score)

17/17 [==============================] - 1s 84ms/step - loss: 2.0028 - accuracy: 0.6109
[9.7202522e-01 1.1304166e-06 1.5874539e-08 1.9208174e-10 1.9409043e-09
 2.8128757e-06 1.5673648e-05 1.5046643e-07 1.2890866e-06 2.7953813e-02]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 